In [2]:
#pip install Sense-Hat

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install speedtest-cli

In [3]:
filename = "speeds.csv"

In [4]:
import os
import speedtest 

In [5]:
import datetime
import time

In [6]:
import socket
import requests
import json

In [7]:
import platform
import socket
import uuid

In [9]:
from sense_hat import SenseHat

ModuleNotFoundError: No module named 'RTIMU'

In [8]:

def ProcessSenseHat():
    try:
        sense = SenseHat()
        
        humidity =  sense.get_humidity()
        temperature = sense.get_temperature()
        tempFromHumidity = sense.get_temperature_from_humidity()
        tempFromPressure =  sense.get_temperature_from_pressure()
        pressure =  sense.get_pressure()
        accel_only = sense.get_accelerometer()
        accelP = accel_only.pitch
        accelR = accel_only.roll
        accelY = accel_only.yaw
        raw = sense.get_accelerometer_raw()
        accelRawX = raw.x
        accelRawY =raw.y
        accelRawZ = raw.z
        compass =sense.get_compass()
        orientation = sense.get_orientation()
        orientationP =orientation.pitch
        orientationR = orientation.roll
        orientationY =orientation.yaw
        gyro_only = sense.get_gyroscope()
        gyroP =gyro_only.pitch
        gyroR =gyro_only.roll
        gyroY =gyro_only.yaw
        data= {
                "humidity": humidity,
                "temperature": temperature,
                "tempFromHumidity": tempFromHumidity,
                "tempFromPressure": tempFromPressure,
                "pressure": pressure,
                "accelP": accelP,
                "accelR": accelR,
                "accelY": accelY,
                "accelRawX": accelRawX,
                "accelRawY": accelRawY,
                "accelRawZ": accelRawZ,
                "compass": compass,
                "orientationP": orientationP,
                "orientationR": orientationR,
                "orientationY": orientationY,
                "gyroP": gyroP,
                "gyroR": gyroR,
                "gyroY": gyroY  
            }  
        url = 'https://alddataapi.azurewebsites.net/api/Sensehats'
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        stringData = json.dumps(data)
        r = requests.post(url, data=json.dumps(d), headers=headers,verify=False)
        print(r)
    except Exception as e:
        print(e)


ModuleNotFoundError: No module named 'RTIMU'

In [8]:
def GetISPInfo():
    ipUrl = "http://ip-api.com/json"
    response= json.loads(requests.get(ipUrl).content)
    print(response)
    print(response["query"])
    return response

In [9]:
def test():
    s = speedtest.Speedtest()
    
    s.get_servers()
    s.get_best_server()
    s.download()
    s.upload()
    res = s.results.dict()
    print(res)
    return res["download"], res["upload"], res["ping"]

In [10]:
hostname = socket.gethostname()
print(hostname)

alandesktop


In [11]:
def ProcessSpeedTest(linesToWrite):
    newId = uuid.uuid4()
    now = datetime.datetime.utcnow()
    datepart = now.strftime("%Y_%m_%d")
    filename = "data/speed." + datepart +".csv"
    print(filename)
    try:
        ip = socket.gethostbyname(socket.gethostname())
        ispInfo = GetISPInfo()
        externalIp = ispInfo["query"]
        print("running test")
        d, u, p = test()
        print("Download was ", round(d/1024/1024,1))
        print(d,u,p,ip,ispInfo)
    except  Exception as e:
        print("exception ",e)
        d = -1
        u = -1
        p = -1
        ip = socket.gethostbyname(socket.gethostname())
    try:
        url = "https://alddataapi.azurewebsites.net/api/Speeds"
        # url = "https://localhost:44323/api/Speeds"
        print(url)

        d = {
            "id": str(newId),  "utc": now.isoformat() ,  "pingMs": int(p),  "downloadSpeed":int(d),  "uploadSpeed": int(u),  
            "publicIp": externalIp,  "isp": "string",  "lat": ispInfo["lat"],  "lon": ispInfo["lon"],  "data": str(json.dumps(ispInfo))
        }

        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        stringData = json.dumps(d)
        print(stringData)
        r = requests.post(url, data=json.dumps(d), headers=headers,verify=False)
        print(r)
    except Exception as e:
        print("Unable to put",e)    
    
    if os.path.isfile(filename):
        print(filename + ' exists')
    else:
        with open(filename, 'w') as f:
            f.write('utc,ip,download,upload,ping\n') 
    with open(filename, 'a') as f:
        line = '{},{},{},{},{}\n'.format(datetime.datetime.utcnow(),ip,d, u, p)
        linesToWrite.append(line)
        print(line)
        
        try:
            for line in linesToWrite:
                print("Writing line",line)
                f.write(line)
            print("wrote lines")
            linesToWrite = []
            
        except:
            print("Unable to write line")
            for line in linesToWrite:
                print (line)
            print("----")
    return linesToWrite

In [ ]:
print("ready to start work")
linesToWrite = []
while (True):    
    try:
        
        linesToWrite = ProcessSpeedTest(linesToWrite)
    except Exception as e:
        print("Exception ",e)
    print('sleeping')
    time.sleep(600) # 60*10 - 10 minutes 

ready to start work
data/speed.2022_04_13.csv
{'status': 'success', 'country': 'United States', 'countryCode': 'US', 'region': 'MN', 'regionName': 'Minnesota', 'city': 'Otsego', 'zip': '55330', 'lat': 45.2741, 'lon': -93.5913, 'timezone': 'America/Chicago', 'isp': 'Verizon Business', 'org': 'Verizon Business', 'as': 'AS6167 Verizon Business', 'query': '174.250.58.7'}
174.250.58.7
running test
{'download': 4784563.659182558, 'upload': 634326.4317355853, 'ping': 113.247, 'server': {'url': 'http://speedtest.twinvalley.net:8080/speedtest/upload.php', 'lat': '39.3486', 'lon': '-97.4533', 'name': 'Miltonvale, KS', 'country': 'United States', 'cc': 'US', 'sponsor': 'Twin Valley Communications', 'id': '6889', 'host': 'speedtest.twinvalley.net:8080', 'd': 180.51476936241065, 'latency': 113.247}, 'timestamp': '2022-04-13T22:04:06.045755Z', 'bytes_sent': 1007616, 'bytes_received': 6352208, 'share': None, 'client': {'ip': '174.250.58.7', 'lat': '37.751', 'lon': '-97.822', 'isp': 'Verizon Wireless'

C:\Users\aland\anaconda3\envs\PythonAndR\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alddataapi.azurewebsites.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [201]>
data/speed.2022_04_13.csv exists
2022-04-13 22:04:39.223127,192.168.43.17,{'id': 'aeb82b3a-6cff-41c8-b309-8e543d145d95', 'utc': '2022-04-13T22:04:04.899723', 'pingMs': 113, 'downloadSpeed': 4784563, 'uploadSpeed': 634326, 'publicIp': '174.250.58.7', 'isp': 'string', 'lat': 45.2741, 'lon': -93.5913, 'data': '{"status": "success", "country": "United States", "countryCode": "US", "region": "MN", "regionName": "Minnesota", "city": "Otsego", "zip": "55330", "lat": 45.2741, "lon": -93.5913, "timezone": "America/Chicago", "isp": "Verizon Business", "org": "Verizon Business", "as": "AS6167 Verizon Business", "query": "174.250.58.7"}'},634326.4317355853,113.247

Writing line 2022-04-13 22:04:39.223127,192.168.43.17,{'id': 'aeb82b3a-6cff-41c8-b309-8e543d145d95', 'utc': '2022-04-13T22:04:04.899723', 'pingMs': 113, 'downloadSpeed': 4784563, 'uploadSpeed': 634326, 'publicIp': '174.250.58.7', 'isp': 'string', 'lat': 45.2741, 'lon': -93.5913, 'data': '{"status": "success", "country":